# OpenMX Interface

The OpenMX Interface module enables seamless conversion from [OpenMX](https://openmx-square.org/) (an open-source DFT software developed primarily at the University of Tokyo) calculation outputs to the DeepH data format. As one of DeepH's earliest collaborative software platforms, OpenMX offers efficient nanoscale material simulations using pseudopotentials and numerical atomic orbitals. This conversion empowers researchers with existing OpenMX datasets to leverage DeepH's machine learning capabilities for accelerated electronic structure calculations while maintaining first-principles accuracy.

**Note**: This converter is designed for OpenMX 3.9 format, with automatic version detection from `openmx.scfout` headers. We welcome community contributions to expand support to additional OpenMX versions.

## File Structure
Assuming you have already read the OpenMX [documentation](https://www.openmx-square.org/openmx_man3.9/index.html) and are familiar with conducting DFT calculations using OpenMX, to prepare your OpenMX data for DeepH training, you must organize the data for different material structures according to the following convention:

``` bash
openmx_dataset/
├── structure_1/
│   ├── openmx.scfout
│   └── openmx.out
├── structure_2/
│   └── ...
└── ...
```

Here, `structure_1`, `structure_2`, etc., represent the names of individual datasets and can be any combination of characters. The DeepH-dock data conversion tool will automatically transform this organized data into the format recognized by DeepH-pack. It will be converted into the DeepH format like:

```bash
deeph_datasets/
├── structure_1/
│   ├── info.json
│   ├── POSCAR
│   ├── hamiltonian.h5
│   ├── overlap.h5
│   ├── density_matrix.h5       # (Optional)
│   └── position_matrix.h5      # (Optional)
├── structure_2/
│   └── ...
└── ...
```

## Begin with Command Line Interface

User can use the command line interface to convert OpenMX format to DeepH format.

```bash
dock convert openmx to-deeph ./openmx_data ./tmp/deeph_data -t 0 -p 2
```

```bash
Data: 2it [00:00, 425.99it/s]
[done] Translation completed successfully!
```

The description of the arguments can be shown by:

```bash
dock convert openmx to-deeph -h
```

```bash
Usage: dock convert openmx to-deeph [OPTIONS] OPENMX_DIR DEEPH_DIR

  Translate the OpenMX output data to DeepH DFT data training set format.

Options:
  --ignore-S                  Do not export overlap.h5
  --ignore-H                  Do not export hamiltonian.h5
  --export-rho                Export density_matrix.h5
  --export-r                  Export position_matrix.h5
  -p, --parallel-num INTEGER  The parallel processing number, -1 for using all
                              of the cores.  [default: -1]
  -t, --tier-num INTEGER      The tier number of the OpenMX source data, -1
                              for [openmx_dir], 0 for
                              <openmx_dir>/<data_dirs>, 1 for
                              <openmx_dir>/<tier1>/<data_dirs>, etc.
                              [default: 0]
  --force                     Force to overwrite the existing files.
  -h, --help                  Show this message and exit.
```

## Advanced Usage: Python Class API

For advanced users, deeph-dock provides the `OpenMXDatasetTranslator` class.

In [1]:
from deepx_dock.convert.openmx.translate_openmx_to_deeph import OpenMXDatasetTranslator

translator = OpenMXDatasetTranslator(
    openmx_data_dir="./openmx_data",
    deeph_data_dir="/tmp/deeph_data",
    n_jobs=2,
)
translator.transfer_all_openmx_to_deeph()

Data: 2it [00:00, 57.31it/s]


In [2]:
from pathlib import Path
print([str(v) for v in list(Path("/tmp/deeph_data").iterdir())])
print([str(v) for v in list(Path("/tmp/deeph_data/MoTe2").iterdir())])

['/tmp/deeph_data/MoTe2', '/tmp/deeph_data/Bi2Se3_SOC']
['/tmp/deeph_data/MoTe2/POSCAR', '/tmp/deeph_data/MoTe2/hamiltonian.h5', '/tmp/deeph_data/MoTe2/overlap.h5', '/tmp/deeph_data/MoTe2/info.json']
